In this exercise, you'll make your first submission to the [**Petals to the Metal**](https://www.kaggle.com/c/tpu-getting-started) competition.  You'll learn how to accept the competition rules, run a notebook on Kaggle that uses (free!) TPUs, and how to submit your results to the leaderboard.

We won't cover the code in detail here, but if you'd like to dive into the details, you're encouraged to check out the [tutorial notebook](https://www.kaggle.com/ryanholbrook/create-your-first-submission).

# Code #

The code reproduces the code we covered together in **[the tutorial](https://www.kaggle.com/ryanholbrook/create-your-first-submission)**.  If you commit the notebook by following the instructions above, then the code is run for you.

## Load Helper Functions ##

In [ ]:
from petal_helper import *

BATCH_SIZE = 12 * strategy.num_replicas_in_sync
print('Batchsize:',BATCH_SIZE)

In [ ]:
def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# %% [markdown]

# %% [code]
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
import tensorflow as tf

# Detect and initialize TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="local")  
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print(f"Running on TPU: {tpu.master()}")
except Exception as e:
    print(f"Could not initialize TPU: {e}")
    strategy = tf.distribute.get_strategy()  # Fallback to default CPU/GPU
    print("Running on CPU or single GPU")

# Print number of available TPU cores
print("Number of devices:", strategy.num_replicas_in_sync, "🚀")

## Create Distribution Strategy ##

## Loading the Competition Data ##

In [ ]:
ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

print("Training:", ds_train)
print ("Validation:", ds_valid)
print("Test:", ds_test)

## 
Explore the Data ##

Try using some of the helper functions described in the **Getting Started** tutorial to explore the dataset.

In [ ]:
print("Number of classes: {}".format(len(CLASSES)))

print("First five classes, sorted alphabetically:")
for name in sorted(CLASSES)[:5]:
    print(name)

print ("Number of training images: {}".format(NUM_TRAINING_IMAGES))

Examine the shape of the data.

In [ ]:
strategy = tf.distribute.TPUStrategy(tpu)
strategy

In [ ]:
print("Training data shapes:")
for image, label in ds_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

In [ ]:
print("Test data shapes:")
for image, idnum in ds_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

Peek at training data.

In [ ]:
one_batch = next(iter(ds_train.unbatch().batch(16)))
display_batch_of_images(one_batch)

## Define Model #

In [ ]:
def lrfn(epoch):
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)



In [ ]:
with strategy.scope():
    #pretrained_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.Xception(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model = tf.keras.applications.mobilenet.MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable=False
    
    model_201 = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
    model_201.compile(
        optimizer='nadam',
        loss = 'sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
    )
    model_201.summary()

In [ ]:
BATCH_SIZE = 12 * strategy.num_replicas_in_sync
print('Batchsize:',BATCH_SIZE)
# Define training epochs for committing/submitting. (TPU on)
EPOCHS = 30
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE 


with strategy.scope():
    history_201 = model_201.fit(
        get_training_dataset(), 
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=30, 
        callbacks=[lr_callback],
        validation_data=None
    )

In [ ]:
# with strategy.scope():

#     feature_extractor = tf.keras.applications.EfficientNetB7(
#                     weights = 'imagenet', include_top = False, input_shape=[*IMAGE_SIZE, 3])
#     feature_extractor.trainable = True
#     model = tf.keras.Sequential([
#         feature_extractor,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax', dtype='float32')
#     ])
    
#     model.compile(optimizer='adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy']
#     )

# model.summary()

In [ ]:
!pip install -U efficientnet


In [ ]:
import efficientnet.tfkeras as efn


with strategy.scope():
    pretrained_model_noisy = efn.EfficientNetB7(weights='noisy-student', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model_noisy.trainable = False 
    
    model_noisy = tf.keras.Sequential([
        pretrained_model_noisy,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
        
    model_noisy.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
    model_noisy.summary()

In [ ]:
with strategy.scope():
    history_noisy = model_noisy.fit(
        get_training_dataset(), 
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=30, 
        callbacks=[lr_callback],
        validation_data=None
    )

# Ensemble

In [ ]:
from sklearn.metrics import f1_score
val_dataset = get_validation_dataset()
images_ds = val_dataset.map(lambda image, label: image)
labels_ds = val_dataset.map(lambda image, label: label).unbatch()
val_labels = next(iter(labels_ds.batch(3712))).numpy() # get everything as one batch
m1 = model_201.predict(images_ds)
m2 = model_noisy.predict(images_ds)
scores = []
for alpha in np.linspace(0,1,100):
    val_probabilities = alpha*m1+(1-alpha)*m2
    val_predictions = np.argmax(val_probabilities, axis=-1)
    scores.append(f1_score(val_labels, val_predictions, labels=range(104), average='macro'))

best_alpha = np.argmax(scores)/100
    
print('Best alpha: ' + str(best_alpha))

# Validation

In [ ]:
from sklearn.metrics import f1_score
val_dataset = get_validation_dataset()
images_ds = val_dataset.map(lambda image, label: image)
labels_ds = val_dataset.map(lambda image, label: label).unbatch()
val_labels = next(iter(labels_ds.batch(3712))).numpy() # get everything as one batch
m1 = model_201.predict(images_ds)
m2 = model_noisy.predict(images_ds)
scores = []
for alpha in np.linspace(0,1,100):
    val_probabilities = alpha*m1+(1-alpha)*m2
    val_predictions = np.argmax(val_probabilities, axis=-1)
    scores.append(f1_score(val_labels, val_predictions, labels=range(104), average='macro'))

best_alpha = np.argmax(scores)/100
    
print('Best alpha: ' + str(best_alpha))

## Test Predictions ##

Create predictions to submit to the competition.

In [ ]:
test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)

alpha = best_alpha

probabilities = (alpha*model_201.predict(test_images_ds)+(1-alpha)*model_noisy.predict(test_images_ds))

predictions = np.argmax(probabilities, axis=-1)
print(predictions)



In [ ]:
print('Generating submission.csv file...')

# Get image ids from test set and convert to integers
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv